In [1]:
#to prevent gpu to collapse it allow using at 50% of total gpu
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# importing usefull transfer learning libraries

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import tensorflow as tf
import numpy as np
import cv2
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
#to check our ploting area for images
x = np.array([0, 6])
y = np.array([0, 250])
plt.plot(x, y)
plt.show()

In [5]:
#changing image size for preprocessing purpose(as inception input size is 299)
image_size = [256, 256]
train_folder = '/content/drive/MyDrive/Dataset/Train'
test_folder = '/content/drive/MyDrive/Dataset/Test'


In [ ]:
Image = Image.open('/content/drive/MyDrive/Dataset/Train/LeafBlast_Disease/shape 1 .jpeg'),


In [6]:
# Importing the inceptionv3 library as shown below and adding the preprocessing layer to the front of inception v3 model from keras
# Here we will be using imagenet weights
resnet = ResNet50(input_shape= image_size + [3], weights='imagenet', include_top=False)
#inception = InceptionV3(input_shape=image_size + [3], weights='imagenet', include_top=False)

94781440/94765736 [==============================] - 0s 0us/step


In [7]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [9]:
#to check files length
folders = glob('/content/drive/MyDrive/Dataset/Train')


In [8]:
# adding input and output layers
x = Flatten()(resnet.output)


In [10]:
#initializing the inner layers by using sigmoid function
prediction = Dense(2, activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [11]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [12]:
# using this to tackle with loss cost and optimizer input weights by comparing the prediction and the loss function
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  #matrix for accuracy
  metrics=['accuracy']
)

DAta Augmentation

In [15]:
#by using the image generator for importing images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dg = ImageDataGenerator(rescale = 1./256, 
                              shear_range =0.2,
                              #zoom_range = 0.2,
                              #horizontal_flip = True
                              )
test_dg = ImageDataGenerator(rescale= 1./256)


In [21]:
# giving the training path to fatch images with classes by targeting its actual sizes
train_cell = train_dg.flow_from_directory('/content/drive/MyDrive/Dataset/Train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1507 images belonging to 2 classes.


In [22]:
# giving the testing path to fatch images with classes by targeting its actual sizes
test_cell = test_dg.flow_from_directory('/content/drive/MyDrive/Dataset/Test',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 767 images belonging to 2 classes.


In [ ]:
#training my model wtih dense last layer and flatting the first layer
history = model.fit_generator(
  train_cell,
  validation_data = test_cell,
   epochs=10
 

)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/10
20/48 [===========>..................] - ETA: 34s - loss: 1.8101 - accuracy: 0.6007